In [116]:
import pandas as pd
from collections import OrderedDict as od
import string

In [117]:
import re

class RegexPatterns:
    @classmethod
    def symbol_pattern(cls, text):
        pattern = r'<[A-Z]> ::='
        return re.findall(pattern, text)

    @classmethod
    def variable_pattern(cls, text):
        pattern = r'([a-z])<([A-Z])>'
        return re.findall(pattern, text)

patterns = RegexPatterns()

In [118]:
def reserved_words_and_counts(csv_df):
    """
    Analisa um DataFrame contendo strings e extrai palavras reservadas, juntamente com suas contagens de caracteres.

    Parâmetros:
    - csv_df (pandas.DataFrame): O DataFrame contendo strings a serem analisadas.

    Retorna:
    lista: Uma lista de dicionários, cada um contendo informações sobre uma palavra reservada, incluindo a própria palavra
        e a contagem de caracteres. O formato do dicionário é o seguinte:
        [
            {"word": str, "size": int},
            {"word": str, "size": int},
            ...
        ]

    Nota:
    - Palavras reservadas são identificadas como strings em minúsculas, excluindo o caractere '<'.
    - A função calcula o tamanho (contagem de caracteres) de cada palavra reservada.
    - A lista retornada contém dicionários com informações sobre cada palavra reservada e seu tamanho.
    """
    reserved_words = [row for row in csv_df.values for char in row if char.islower() and char != "<"]         
    size = [len(c) for word in reserved_words for c in word]
    
    json_data = []
    for word, size in zip(reserved_words, size):
        json = {
            "word": word,
            "size": size
        }
        json_data.append(json)
    
    return json_data


[array(['se'], dtype=object), array(['entao'], dtype=object), array(['senao'], dtype=object)]


In [119]:
def extract_terminals(csv_df):
    """
    Extrai as letras terminais das palavras reservadas obtidas ao analisar um DataFrame contendo strings.

    Parâmetros:
    - csv_df (pandas.DataFrame): O DataFrame contendo strings a serem analisadas.

    Retorna:
    lista: Uma lista de letras terminais individuais obtidas das palavras reservadas.

    Nota:
    - Esta função depende da função 'reserved_words_and_counts' para identificar palavras reservadas e suas contagens.
    - A função extrai letras individuais das palavras reservadas para criar uma lista de letras terminais.
    """
    reserved_words = [row["word"] for row in reserved_words_and_counts(csv_df)]
    
    terminal_letters = [c for word in reserved_words for char in word for c in char]
    return terminal_letters

In [120]:
def unique_terminal_letters(csv_df):
    """
    Extrai letras terminais únicas de um DataFrame contendo strings.

    Parâmetros:
    - csv_df (pandas.DataFrame): O DataFrame contendo strings a serem analisadas.

    Retorna:
    lista: Uma lista de letras terminais únicas encontradas nas strings.

    Nota:
    - A função ignora letras maiúsculas, 'ε' (épsilon) e caracteres não alfabéticos.
    - Utiliza o 'collections.OrderedDict' para preservar a ordem das letras terminais únicas.
    """
    terminal_letters = list(od.fromkeys((c for row in csv_df.values for char in row for c in char if c.islower() and c != 'ε')))
    return terminal_letters

In [121]:
def create_afnd_skeleton(csv_df):
    """
    Cria o esqueleto de um AFND (Autômato Finito Não Determinístico) representado como um DataFrame.

    Parâmetros:
    - csv_df (pandas.DataFrame): O DataFrame contendo strings a serem analisadas.

    Retorna:
    pandas.DataFrame: O esqueleto do AFND representado como um DataFrame com cabeçalhos de coluna apropriados.

    Nota:
    - O DataFrame inclui colunas para o alfabeto ('sigma') e letras terminais.
    - As linhas representam estados no AFND.
    - A primeira linha é o estado inicial ('S'), e as linhas subsequentes representam estados rotulados com letras maiúsculas.
    - Células vazias indicam transições que não estão definidas.
    """
    terminal_letters = unique_terminal_letters(csv_df)
    afnd_skeleton_df = pd.DataFrame(columns=['sigma'] + [str(c) for c in terminal_letters])
    afnd_skeleton_df.at[0, 'sigma'] = 'S'
    
    alphabet = list(string.ascii_uppercase)
    size = len(extract_terminals(csv_df))
    
    symbols = [symbol for letters in alphabet[:size] for symbol in letters]
    for i, symbol in enumerate(symbols):
        afnd_skeleton_df.at[i+1, "sigma"] = symbol
    
    afnd_skeleton_df = afnd_skeleton_df.fillna('')
    
    return afnd_skeleton_df

In [122]:
import re

def extract_variables_df(csv_df):
    """
    Extrai informações sobre variáveis de um DataFrame contendo strings.

    Parâmetros:
    - csv_df (pandas.DataFrame): O DataFrame contendo strings a serem analisadas.

    Retorna:
    lista: Uma lista de dicionários, cada um contendo informações sobre uma variável, incluindo o símbolo,
        terminais associados e variáveis associadas. O formato do dicionário é o seguinte:
        [
            {"symbol": str, "terminals": list, "variables": list},
            {"symbol": str, "terminals": list, "variables": list},
            ...
        ]

    Nota:
    - A função utiliza expressões regulares para encontrar padrões de símbolos e variáveis nas strings do DataFrame.
    - O resultado é uma lista de dicionários contendo informações sobre cada variável encontrada.
    """
    
    symbol_pattern = r'<[A-Z]> ::='
    variable_pattern = r'([a-z])<([A-Z])>'
    
    lines = [str(row) for row in csv_df.values if re.search(symbol_pattern, str(row)) for char in row]
    
    rg = []
    
    for line in lines:       
        symbol = [match[1] for match in re.findall(symbol_pattern, line) if re.search(variable_pattern, line)]
        terminals = [match[0] for match in re.findall(variable_pattern, line)]
        variables = [match[1] for match in re.findall(variable_pattern, line)]
        
        json = {
            "symbol": symbol,
            "terminals": terminals,
            "variables": variables
        }
        rg.append(json)
    return rg

In [123]:
import re

def extract_variables_list(words):
    """
    Extrai informações sobre variáveis de uma lista de palavras.

    Parâmetros:
    - words (list): A lista de palavras a serem analisadas.

    Retorna:
    lista: Uma lista de dicionários, cada um contendo informações sobre uma variável, incluindo o símbolo,
        terminais associados e variáveis associadas. O formato do dicionário é o seguinte:
        [
            {"symbol": str, "terminals": list, "variables": list},
            {"symbol": str, "terminals": list, "variables": list},
            ...
        ]

    Nota:
    - A função utiliza expressões regulares para encontrar padrões de símbolos e variáveis na lista de palavras.
    - Os símbolos são identificados com base nos valores adjacentes à ocorrência do "::=" na lista.
    - Os resultados são organizados em uma lista de dicionários, cada um representando uma variável.
    """
    
    variable_pattern = r'([a-z])<([A-Z])>'
    
    symbols = [re.sub(r'[^a-zA-Z]', '', words[i - 1]) for i, value in enumerate(words) if value == '::=']
    
    result_lists = []
    current_list = []
    collect_values = False

    # Iterate through the input list
    for item in words:
        # Check if "[" is found, and stop collecting values if so
        if "[" in item:
            if current_list:
                result_lists.append(current_list)
                current_list = []
            collect_values = False
        # If collect_values is True and the item is not at index 0 or 1, remove "[" and "]" and add the item to the current_list
        if collect_values and words.index(item) not in [0, 1]:
            item = item.strip("[]")
            current_list.append(item)
        # Check if item is at index 1, and start collecting values if so
        if item == "::=":
            collect_values = True

    # Append the last current_list if it is not empty
    if current_list:
        result_lists.append(current_list)

    rg = []
    
    for i, symbol in enumerate(symbols):
        terminals = []
        variables = []
        for j, word in enumerate(result_lists[i]):
            if re.search(variable_pattern, word) is None:
                continue
            terminals.append([match[0] for match in re.findall(variable_pattern, word) if re.search(variable_pattern, word) is not None])
            variables.append([match[1] for match in re.findall(variable_pattern, word) if re.search(variable_pattern, word) is not None])
            (variables)
            
        json = {
            "symbol": symbol,
            "terminals": terminals,
            "variables": variables
        }
        rg.append(json)   
    return rg

In [124]:
import re

def replace_variables(afnd_df, csv_df, last_state):
    """
    Substitui as variáveis em um AFND (Autômato Finito Não Determinístico) representado como um DataFrame.

    Parâmetros:
    - afnd_df (pandas.DataFrame): O DataFrame representando o AFND.
    - csv_df (pandas.DataFrame): O DataFrame contendo strings com definições de variáveis.
    - last_state (str): O último estado no AFND antes da substituição.

    Retorna:
    tuple: Uma tupla contendo o DataFrame atualizado do AFND e uma lista de palavras modificadas.

    Nota:
    - A função substitui as variáveis nos estados do AFND com novos símbolos, começando após o último estado original.
    - Os novos símbolos são gerados com base nas letras do alfabeto em ordem.
    - A função preserva a estrutura original das palavras, substituindo as variáveis correspondentes.
    - A lista de palavras modificadas é retornada junto com o DataFrame atualizado.
    """
    
    alphabet = list(string.ascii_uppercase)
    variable_pattern = r'([a-z])<([A-Z])>'
    
    old_variables = list(od.fromkeys(variable for line in extract_variables_df(csv_df) for variable in line["variables"]))
    new_states_count = len(old_variables)
    index_last_state = alphabet.index(last_state) + 1
    total = new_states_count + index_last_state
    symbols = [symbol for letters in alphabet[index_last_state:total] for symbol in letters]

    # Update the "sigma" column in afnd_df starting from the end
    for i, symbol in enumerate(symbols, start=index_last_state):
        afnd_df.at[i+1, "sigma"] = symbol

    lines = [str(row) for row in csv_df.values if re.search(variable_pattern, str(row)) for char in row]
    words_original = [word for sentence in lines for word in sentence.split()]
    words = words_original.copy()

    new_variables = []

    for i in range(new_states_count):
        for j, word in enumerate(words):
            match = re.search(variable_pattern, word)
             
            if match is None:
                continue
                
            if match.group(2) == old_variables[i]:
                words[j] = f"{match[1]}<{symbols[i]}>"
                json_data = {
                    "old_variable": old_variables[i],
                    "new_variable": symbols[i]
                }
                new_variables.append(json_data)
    
    
    def generate_same_structure(original_structure, new_structure):
        result_structure = []

        for orig, new in zip(original_structure, new_structure):
            if isinstance(orig, list):
                result_structure.append(generate_same_structure(orig, new))
            else:
                # Preserve the original square brackets
                result_structure.append(orig[0] + new[1:-1] + orig[-1])

        return result_structure
    
    words = generate_same_structure(words_original, words)
    words = [word.replace('||', '|') for word in words]
    for data in new_variables:
        words = [word.replace(f'<{data["old_variable"]}>', f'<{data["new_variable"]}>') for word in words]
                
    new_variables = [dict(t) for t in {tuple(d.items()) for d in new_variables}]
    new_variables = sorted(new_variables, key=lambda k: k['new_variable'])
    
    afnd_df = afnd_df.fillna('')
    
    return afnd_df, words

In [125]:
def insert_value_df(df, search_column, search_row, target_column, value):
    """ Esta função insere um valor em uma célula de um DataFrame, ou concatena o valor com o valor existente na célula.

    Args:
        df (pandas.DataFrame): O DataFrame a ser modificado
        search_column (str): O nome da coluna a ser pesquisada
        search_row (str): O valor da linha a ser pesquisada
        target_column (str): O nome da coluna onde o valor será inserido ou concatenado
        value (str): O valor a ser inserido ou concatenado  

    Returns:
        df: O DataFrame modificado
    
    """
    if value == '':
        df.fillna('', inplace=True)
        return df
    if df.loc[df[search_column] == search_row, target_column].any() == False:
        df.loc[df[search_column] == search_row, target_column] = value
        return df
    df.loc[df[search_column] == search_row, target_column] += f"{','}{value}"
        
    return df

In [126]:
def populate_variables(afnd_df, words):
    """
    Preenche o AFND (Autômato Finito Não Determinístico) com informações sobre variáveis.

    Parâmetros:
    - afnd_df (pandas.DataFrame): O DataFrame representando o AFND.
    - words (list): A lista de palavras contendo informações sobre variáveis.

    Retorna:
    pandas.DataFrame: O DataFrame do AFND atualizado com informações sobre variáveis.

    Nota:
    - A função extrai informações sobre variáveis a partir da lista de palavras e popula as células correspondentes no AFND.
    - Os dados são preenchidos nas células do AFND de acordo com as informações sobre variáveis encontradas.
    """
    new_variables = extract_variables_list(words)
    
    for new_variable in new_variables:
        for i, terminals, variables in zip(range(len(new_variable["terminals"])), new_variable["terminals"], new_variable["variables"]):
            for terminal, variable in zip(terminals, variables):   
                afnd_df = insert_value_df(afnd_df, 'sigma', new_variable["symbol"], terminal, variable)
    return afnd_df

In [127]:
def create_afnd(csv_df):
    """
    Cria um AFND (Autômato Finito Não Determinístico) representado como um DataFrame.

    Parâmetros:
    - csv_df (pandas.DataFrame): O DataFrame contendo strings a serem analisadas.

    Retorna:
    pandas.DataFrame: O DataFrame representando o AFND gerado.

    Nota:
    - A função cria o esqueleto do AFND utilizando a função 'create_afnd_skeleton'.
    - Em seguida, preenche o AFND com transições com base nas palavras reservadas e variáveis do DataFrame de entrada.
    """

    afnd_df = create_afnd_skeleton(csv_df) 
    
    reserved_counts = reserved_words_and_counts(csv_df)  
    alphabet = list(string.ascii_uppercase)
    last_state = 'S'
    final_state = []
    flag = False
    
    for row in reserved_counts: 
        
        initial_state = 'S'
        
        word = [c for char in row["word"] for c in char]
        
        for index, char in enumerate(word): 
            if not final_state: # Se não for um estado final
                if not flag: # Se estiver vazio, não tem estado final no até o fim da primeira palavra
                    afnd_df.loc[afnd_df['sigma'] == initial_state, char] = alphabet[index]
                    last_state = alphabet[index] 
                    flag = True
                    continue 
                  
                if flag and last_state == alphabet[0]:
                    afnd_df.loc[afnd_df['sigma'] == last_state, char] = alphabet[1]   
                    last_state = alphabet[1]           
            else:      
                if last_state in (final_state):
                    if not afnd_df.loc[afnd_df['sigma'] == last_state, char].isna().all():
                        afnd_df.loc[afnd_df['sigma'] == initial_state, char] += f"{',' if afnd_df.loc[afnd_df['sigma'] == initial_state, char].any() else ''}{alphabet[alphabet.index(last_state) + 1]}"
                        last_state = alphabet[((alphabet.index(last_state) + 1))]
                    continue
                afnd_df = insert_value_df(afnd_df, 'sigma', last_state, char, alphabet[((alphabet.index(last_state) + 1))])
                last_state = alphabet[((alphabet.index(last_state) + 1))] 
                      
        final_state.append(last_state)
    afnd_df, words = replace_variables(afnd_df, csv_df, last_state)
    
    print(words)
    
    afnd_df = populate_variables(afnd_df, words)
    return afnd_df

csv_df = pd.read_csv('./entrada.csv',  header=None)
csv_df

afnd_df = create_afnd(csv_df)
afnd_df

[array(['se'], dtype=object), array(['entao'], dtype=object), array(['senao'], dtype=object)]
[array(['se'], dtype=object), array(['entao'], dtype=object), array(['senao'], dtype=object)]
["['<S>", '::=', 'a<M>', '|', 'e<M>', '|', 'i<M>', '|', 'o<M>', '|', 'u<M]', "['<M>", '::=', 'a<M>', '|', 'e<M>', '|', 'i<M>', '|', 'o<M>', '|', 'u<M>', '|', "ε']"]


,sigma,s,e,n,t,a,o,i,u
0,S,"A,H","C,M",,,M,M,M,
1,A,,B,,,,,,
2,B,,,,,,,,
3,C,,,D,,,,,
4,D,,,,E,,,,
5,E,,,,,F,,,
6,F,,,,,,G,,
7,G,,,,,,,,
8,H,,I,,,,,,
9,I,,,J,,,,,


In [128]:
def determinize_afnd(csv_df, afnd_df):
    """
    Preenche uma linha do AFND (Autômato Finito Determinístico) com o símbolo de indeterminismo. Juntamente com os estados de cada transição.
    
    Parâmetros:
    - csv_df (pandas.DataFrame): O DataFrame contendo strings a serem analisadas.
    - afnd_df (pandas.DataFrame): O DataFrame representando o AFND gerado.
    - indeterminism_symbol (str): O símbolo de indeterminismo a ser utilizado.
    
    Retorna:
    pandas.DataFrame: O DataFrame do AFD atualizado com o símbolo de indeterminismo.
    """
    indeterminisms = [(afnd_df.loc[afnd_df[terminal].str.len() > 1, terminal]).tolist() for terminal in afnd_df.columns[1:] if not afnd_df.loc[afnd_df[terminal].str.len() > 1, terminal].empty] # Lista de listas contendo os indeterminismos iniciais do AFND
    
    all_indeterminisms = indeterminisms.copy()
    
    columns = unique_terminal_letters(csv_df)
    
    while indeterminisms:
        for indeterminism in indeterminisms:
            indeterminism_symbol = indeterminism[0]
            last_index = afnd_df['sigma'].index[-1]
            afnd_df.at[last_index + 1, "sigma"] = indeterminism_symbol 
             
            for state in afnd_df['sigma'].iloc[-1].split(','): # Percorre os estados do indeterminismo
                for column in columns: # Percorre as colunas do AFND
                    afnd_df = insert_value_df(afnd_df, 'sigma', indeterminism_symbol, column, afnd_df.loc[afnd_df['sigma'] == state, column].iloc[0])
            indeterminisms.pop(0)
                    
        last_row_df = afnd_df.iloc[-1].to_frame().T # Seleciona a última linha do AFND
        new_indeterminisms = [(last_row_df.loc[last_row_df[terminal].str.len() > 1, terminal]).tolist() for terminal in last_row_df.columns[1:] if not last_row_df.loc[last_row_df[terminal].str.len() > 1, terminal].empty] # Seleciona os indeterminismos da última linha do AFND
        indeterminisms.extend(new_indeterminisms) # Adiciona os novos indeterminismos à lista de indeterminismos
         
        all_indeterminisms.extend(new_indeterminisms) # Lista de listas contendo todos os indeterminismos do AFD
    
        for value_list in all_indeterminisms:
            value = value_list[0]
            value_without_comma = value.replace(',', '')
            
            for col in afnd_df.columns:
                # Use the apply method to apply the transformation to each cell
                afnd_df[col] = afnd_df[col].apply(lambda x: f'[{value_without_comma}]' if x == value else x) 
    return afnd_df

afd = determinize_afnd(csv_df, afnd_df)
afd
    

,sigma,s,e,n,t,a,o,i,u
0,S,[AH],[CM],,,M,M,M,
1,A,,B,,,,,,
2,B,,,,,,,,
3,C,,,D,,,,,
4,D,,,,E,,,,
5,E,,,,,F,,,
6,F,,,,,,G,,
7,G,,,,,,,,
8,H,,I,,,,,,
9,I,,,J,,,,,
